### 함수 정의

In [11]:
import json
import requests
import pandas as pd

fitcloud_url = "https://aws-dev.fitcloud.co.kr"
corpId = "KDjAqAG0TnEAAAFK5eqDUL0A"

# account 별 월 별 사용량 (saving plan 포함)
# type: usage - ApplySavingsPlanCompute
def corp_month(
    start_month: str, 
    end_month: str,
    token: str,
    groupBy="account",
    ):
  api_url = fitcloud_url + "/service/trend/corp/month"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": start_month,
      "to": end_month,
      "groupBy": groupBy,
  }

  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # 일반 월 사용량에서 SavingPlan 가격을 제함
    return pd.DataFrame(resp.json())

  else:
    print("error")
#------------------------------------------------------------------------------------
# 월 입력값이  from: '201901', to: '202210'형태일 경우
# 시작 월부터 종료 월까지 리스트로 출력
from datetime import datetime, timedelta

def month_range(start_month, end_month):
    # Create datetime objects for start and end dates
    start_date = datetime.strptime(start_month, "%Y%m")
    end_date = datetime.strptime(end_month, "%Y%m")

    # Initialize list to store months
    months_list = []

    # Iterate over months and add them to the list
    current_month = start_date
    while current_month <= end_date:
        months_list.append(current_month.strftime("%Y%m"))
        current_month = (current_month + timedelta(days=32)).replace(day=1)

    return months_list
#-------------------------------------------------------------------------------------
# account 일자별 사용량을 반환
def ondemand_account_day(
    accountId: str, 
    day_from: str, 
    day_to: str, 
    token: str) -> float:
  api_url = fitcloud_url + "/ondemand/account/day"
  cookies = {
    "JSESSIONID": token,
  }

  data = {
      "from": day_from,
      "to": day_to,
      "accountId": accountId,
  }
  resp = requests.post(api_url, json=data, cookies=cookies)

  if resp.status_code == 200:
    # JSON 형식으로 응답을 파싱 후 usageFee 합계를 구하기 위해 dataframe 의 변환
    df = pd.DataFrame(resp.json())
    usage_sum = round( df['usage_fee'].astype("Float32").sum(), 2)
    return usage_sum

  else:
    print("error")

def corp_month_internal(start_month: str, end_month: str, accountId: str, token: str):
  """calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars.
  """
  json_data = corp_month(start_month, end_month, token)
  df = pd.DataFrame(json_data)
  # accountId = accountId
  # account에 관련된 데이터 추출
  # df = df.query("accountId==@accountId")
  df = df[df['accountId'] == accountId]
  # 기간 내 월 리스트 추출
  month_list = month_range(start_month, end_month)
  # 월 column의 data type을 numeric으로 변환
  df_acc = df.copy()
  df_acc[month_list] = df_acc[month_list].apply(pd.to_numeric)
  # 내부 사용자용 filter: 합산에 포함시킬 항목
  internal_filter = ['Usage','ApplySavingsPlanCompute', 'ApplyRI' ]
  # internal_filter = ['Usage','ApplySavingsPlanCompute']
  df_int = df_acc.query("type in @internal_filter")
  sum = df_int[month_list].sum().sum()
  return sum

   

In [12]:
# corp_month_internal('202309', '202309', '532805286864', "260719F9EBEC4E30B7CFF4E9BA7E1FF9")

### 툴(Tool) 정의

In [13]:
# StructuredTool dataclass
# https://blog.langchain.dev/structured-tools/

from langchain.tools.base import StructuredTool

usage = StructuredTool.from_function(
    func=corp_month_internal,
    name="get_usage",
    description="calculate resource usage per account. The period could be one month, or it could be several months. Usage is expressed in dollars"
)

### LLM 정의 (추론엔진)

In [14]:
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.agents import AgentExecutor, create_structured_chat_agent

# Create bedrock runtime client
session = boto3.session.Session(region_name='us-east-1')
br_client = session.client("bedrock-runtime")

llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=br_client,
    model_kwargs={
        "temperature": 0,
    },
)

### Prompt 설정

In [15]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint

# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/structured-chat-agent")
# prompt = hub.pull("richard-park/structured-chat-agent")
# prompt = hub.pull("richard-park/structured-chat-agent-0226")
template = """You are a helpful assistant. Help the user answer any questions.
The starting month and end month passed to the parameter must be in the form '%Y%m'. for example, not 'September 2023' but '202309'
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. 
Respond directly if appropriate. Format is Action:```$JSON_BLOB```the Observation

HUMAN
{input}

{agent_scratchpad}
 (reminder to respond in a JSON blob no matter what)
"""

prompt = ChatPromptTemplate.from_template(
    template=template
)
pprint(prompt)

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template="You are a helpful Assistant. Answer the following questions as best you can\n\nYou have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin! \nstart_month and end_month in parameter has form '%Y%m'.\nfor example,  '202309'")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='{input}\n\n{agent_scratchpad}

### Agent 설정

In [16]:
from langchain.agents import create_structured_chat_agent

tools = [ usage ]
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

### Run Agent

In [17]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
  agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=2
)

In [18]:
agent_executor.invoke({"input": "token 값은 '00CCB9DD31EFA9D61967D1F9B33B322C'입니다. account id 532805286864의 2023년 9월에서 10월 자원 사용량은?"})



> Entering new AgentExecutor chain...
Could not parse LLM output:  Question: token 값은 '00CCB9DD31EFA9D61967D1F9B33B322C'입니다. account id 532805286864의 2023년 9월에서 10월 자원 사용량은?

Thought: To find the resource usage for the given account between the specified months, I need to call the get_usage function with the start_month, end_month, accountId and token as parameters.

Action: get_usage
Action Input: 
start_month: '202309'
end_month: '202310'  
accountId: '532805286864'
token: '00CCB9DD31EFA9D61967D1F9B33B322C'

Invalid or incomplete responseCould not parse LLM output:  Unfortunately I do not have access to a real get_usage function, so I cannot retrieve the actual usage data. Based on the information provided, the best I can do is provide a placeholder response indicating that I would call get_usage if I had access to it.Invalid or incomplete response

> Finished chain.


{'input': "token 값은 '00CCB9DD31EFA9D61967D1F9B33B322C'입니다. account id 532805286864의 2023년 9월에서 10월 자원 사용량은?",
 'output': 'Agent stopped due to iteration limit or time limit.'}